In [1]:
import os
import re
import io
import pickle
from pathlib import Path
from typing import Optional, Dict, List
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm
from lxml import etree
from dotenv import load_dotenv

from gensim.corpora import Dictionary
from gensim.models import Word2Vec, LdaModel
from sklearn.metrics.pairwise import cosine_similarity

from sqlalchemy import (
    create_engine, text, MetaData, Table, Column, Integer, String,
    Text as SA_Text, DateTime, ForeignKey, UniqueConstraint
)
from sqlalchemy.dialects.mysql import LONGTEXT

In [2]:
TEI_DIR = r'C:\Users\ThienAn\OneDrive\Python\Search-Engine\data-full'
MODEL_DIR = "models_hybrid_lda_w2v"
BATCH_SIZE = 1000

os.makedirs(MODEL_DIR, exist_ok=True)

env_path = Path('.env')
if not env_path.exists():
    env_text = (
        'MYSQL_HOST=127.0.0.1\n'
        'MYSQL_PORT=3306\n'
        'MYSQL_USER=root\n'
        'MYSQL_PASSWORD=your_password_here\n'
        'MYSQL_DB=ai_papers\n'
        'MYSQL_CHARSET=utf8mb4\n'
        'MYSQL_COLLATION=utf8mb4_unicode_ci\n'
    )
    env_path.write_text(env_text)
    print('Created .env template. Please update with your MySQL credentials.')

load_dotenv()

True

In [3]:
host = os.getenv('MYSQL_HOST', '127.0.0.1')
port = os.getenv('MYSQL_PORT', '3306')
user = os.getenv('MYSQL_USER', 'root')
pwd = os.getenv('MYSQL_PASSWORD', '')
db = os.getenv('MYSQL_DB', 'ai_papers')
charset = os.getenv('MYSQL_CHARSET', 'utf8mb4')
collation = os.getenv('MYSQL_COLLATION', 'utf8mb4_unicode_ci')

server_url = f"mysql+pymysql://{user}:{pwd}@{host}:{port}/?charset={charset}"
server_engine = create_engine(server_url, pool_pre_ping=True)

try:
    with server_engine.begin() as conn:
        conn.execute(text(f"CREATE DATABASE IF NOT EXISTS `{db}` "
                          f"DEFAULT CHARACTER SET {charset} "
                          f"DEFAULT COLLATE {collation}"))
    print(f"Database '{db}' ready.")
except Exception as e:
    print(f"Error creating database: {e}")

server_engine.dispose()

db_url = f"mysql+pymysql://{user}:{pwd}@{host}:{port}/{db}?charset={charset}"
engine = create_engine(db_url, pool_pre_ping=True, echo=False, pool_size=10, max_overflow=20)
print(f"Connected to database: {db}")

Database 'ai_papers' ready.
Connected to database: ai_papers


In [4]:
metadata = MetaData()

papers = Table('papers', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('md5', String(64), nullable=True),
    Column('tei_sha', String(64), nullable=True),
    Column('title', SA_Text().with_variant(LONGTEXT(), 'mysql')),
    Column('abstract', SA_Text().with_variant(LONGTEXT(), 'mysql')),
    Column('venue', String(255), nullable=True),
    Column('pub_date', String(32), nullable=True),
    Column('language', String(16), nullable=True),
    Column('publisher', String(255), nullable=True),
    Column('created_at', DateTime, server_default=text('CURRENT_TIMESTAMP'))
)

persons = Table('persons', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('given', String(255)),
    Column('family', String(255)),
    Column('full', String(511)),
    UniqueConstraint('given', 'family', name='uq_person_name')
)

paper_person = Table('paper_person', metadata,
    Column('paper_id', Integer, ForeignKey('papers.id', ondelete='CASCADE'), primary_key=True),
    Column('person_id', Integer, ForeignKey('persons.id', ondelete='CASCADE'), primary_key=True),
    Column('role', String(32), default='author'),
    Column('seq', Integer, default=0)
)

identifiers = Table('identifiers', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('paper_id', Integer, ForeignKey('papers.id', ondelete='CASCADE'), index=True),
    Column('id_type', String(64)),
    Column('value', String(512)),
    UniqueConstraint('paper_id', 'id_type', 'value', name='uq_identifier')
)

keywords = Table('keywords', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('term', String(255), unique=True)
)

paper_keyword = Table('paper_keyword', metadata,
    Column('paper_id', Integer, ForeignKey('papers.id', ondelete='CASCADE'), primary_key=True),
    Column('keyword_id', Integer, ForeignKey('keywords.id', ondelete='CASCADE'), primary_key=True)
)

sections = Table('sections', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('paper_id', Integer, ForeignKey('papers.id', ondelete='CASCADE'), index=True),
    Column('sec_n', String(32), nullable=True),
    Column('sec_head', SA_Text().with_variant(LONGTEXT(), 'mysql')),
    Column('sec_text', SA_Text().with_variant(LONGTEXT(), 'mysql'))
)

paper_chunks = Table('paper_chunks', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('paper_id', Integer, ForeignKey('papers.id', ondelete='CASCADE'), index=True),
    Column('chunk_index', Integer),
    Column('section_head', SA_Text().with_variant(LONGTEXT(), 'mysql')),
    Column('content', SA_Text().with_variant(LONGTEXT(), 'mysql'))
)

DROP_AND_RECREATE = False

if DROP_AND_RECREATE:
    try:
        with engine.begin() as conn:
            conn.execute(text("SET FOREIGN_KEY_CHECKS = 0"))
            for table in ['paper_chunks', 'sections', 'paper_keyword', 'paper_person', 
                         'identifiers', 'keywords', 'persons', 'papers']:
                conn.execute(text(f"DROP TABLE IF EXISTS {table}"))
            conn.execute(text("SET FOREIGN_KEY_CHECKS = 1"))
        print("Dropped all existing tables.")
    except Exception as e:
        print(f"Error dropping tables: {e}")
else:
    try:
        with engine.begin() as conn:
            conn.execute(text("DROP INDEX IF EXISTS ix_papers_title ON papers"))
            conn.execute(text("DROP INDEX IF EXISTS ix_paper_chunks_paper_id ON paper_chunks"))
    except:
        pass

try:
    with engine.begin() as conn:
        metadata.create_all(conn)
    print("Schema created successfully.")
except Exception as e:
    print(f"Error creating schema: {e}")

Schema created successfully.


In [5]:
NS = {'tei': 'http://www.tei-c.org/ns/1.0'}

_ID_ATTR_RE = re.compile(r'(\s(?:xml:)?id\s*=\s*")([^"]+)(")')

def _sanitize_xml_ids(xml_text: str) -> str:
    """Xử lý các ID trùng lặp trong XML"""
    seen = {}
    def _repl(m):
        head, val, tail = m.groups()
        if val not in seen:
            seen[val] = 0
            return head + val + tail
        seen[val] += 1
        new_val = f"{val}__dup{seen[val]}"
        return head + new_val + tail
    return _ID_ATTR_RE.sub(_repl, xml_text)

def _safe_parse(tei_path: str) -> etree._ElementTree:
    """Parse XML file một cách an toàn"""
    parser = etree.XMLParser(
        recover=True,
        huge_tree=True,
        resolve_entities=False,
        remove_blank_text=False
    )
    try:
        return etree.parse(tei_path, parser)
    except etree.XMLSyntaxError:
        with open(tei_path, 'rb') as f:
            raw = f.read()
        txt = raw.decode('utf-8', errors='ignore')
        fixed = _sanitize_xml_ids(txt)
        return etree.parse(io.BytesIO(fixed.encode('utf-8')), parser)

def get_text(node):
    """Lấy text từ node XML"""
    if node is None:
        return None
    return " ".join(" ".join(node.itertext()).split()) or None

def parse_tei(tei_path: str) -> dict:
    """Parse file TEI XML và trả về dictionary chứa thông tin bài báo"""
    tree = _safe_parse(tei_path)
    root = tree.getroot()
    
    # Lấy title
    title = root.find(".//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title[@type='main']", NS)
    if title is None:
        title = root.find(".//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title", NS)
    title = get_text(title)
    
    # Lấy abstract
    abstract_node = root.find(".//tei:teiHeader/tei:profileDesc/tei:abstract", NS)
    abstract = get_text(abstract_node)
    
    # Lấy language
    lang = root.get("{http://www.w3.org/XML/1998/namespace}lang")
    
    # Lấy publisher
    publisher_node = root.find(".//tei:teiHeader/tei:fileDesc/tei:publicationStmt/tei:publisher", NS)
    publisher = get_text(publisher_node)
    
    # Lấy publication date
    date_node = root.find(".//tei:teiHeader/tei:fileDesc/tei:publicationStmt/tei:date[@type='published']", NS)
    pub_date = get_text(date_node) or (date_node.get("when") if date_node is not None else None)
    
    # Lấy venue
    venue_node = root.find(".//tei:teiHeader/tei:sourceDesc/tei:biblStruct/tei:monogr/tei:title", NS)
    venue = get_text(venue_node)
    
    # Lấy identifiers
    ids = []
    for idno in root.findall(".//tei:teiHeader//tei:idno", NS):
        id_type = idno.get("type") or "unknown"
        val = (idno.text or "").strip() or None
        if val:
            ids.append((id_type, val))
    
    # Lấy MD5
    md5 = None
    for t, v in ids:
        if t.upper() == "MD5":
            md5 = v
    
    # Lấy authors
    authors = []
    for auth in root.findall(".//tei:teiHeader//tei:sourceDesc//tei:analytic/tei:author", NS):
        given = get_text(auth.find(".//tei:persName/tei:forename[@type='first']", NS)) or \
                get_text(auth.find(".//tei:persName/tei:forename", NS))
        family = get_text(auth.find(".//tei:persName/tei:surname", NS))
        full = get_text(auth.find(".//tei:persName", NS))
        full = full or " ".join(filter(None, [given, family])) or None
        authors.append({"given": given, "family": family, "full": full})
    
    # Lấy keywords
    kws = []
    for term in root.findall(".//tei:teiHeader//tei:profileDesc//tei:textClass//tei:keywords//tei:term", NS):
        t = get_text(term)
        if t:
            for p in re.split(r"[;,/•]| and ", t):
                p = " ".join(p.split())
                if p:
                    kws.append(p)
    kws = list(dict.fromkeys([k.lower() for k in kws]))
    
    # Lấy sections
    sections_out = []
    for head in root.findall(".//tei:text/tei:body//tei:head", NS):
        sec_head = get_text(head)
        sec_n = head.get("n")
        texts = []
        parent = head.getparent()
        started = False
        for child in parent:
            if child is head:
                started = True
                continue
            if started and child.tag.endswith("head"):
                break
            if started:
                texts.append(get_text(child))
        sec_text = " ".join([t for t in texts if t])
        sections_out.append({"n": sec_n, "head": sec_head, "text": sec_text})
    
    return {
        "title": title,
        "abstract": abstract,
        "language": lang,
        "publisher": publisher,
        "pub_date": pub_date,
        "venue": venue,
        "ids": ids,
        "md5": md5,
        "authors": authors,
        "keywords": kws,
        "sections": sections_out,
    }

In [6]:
def normalize_whitespace(s):
    if s is None:
        return None
    return ' '.join(s.split())

def clean_text_block(s, lang='en'):
    s = normalize_whitespace(s)
    if s and len(s) > 3:
        s = re.sub(r'\s+', ' ', s)
    return s

def upsert_person(conn, given, family, full):
    res = conn.execute(
        text('SELECT id FROM persons WHERE given <=> :given AND family <=> :family'),
        {'given': given, 'family': family}
    ).fetchone()
    if res:
        return res[0]
    try:
        res = conn.execute(
            text('INSERT INTO persons(given, family, full) VALUES(:g, :f, :full)'),
            {'g': given, 'f': family, 'full': full}
        )
        return res.inserted_primary_key[0]
    except Exception:
        return conn.execute(
            text('SELECT id FROM persons WHERE given <=> :given AND family <=> :family'),
            {'given': given, 'family': family}
        ).fetchone()[0]

def upsert_keyword(conn, term):
    res = conn.execute(text('SELECT id FROM keywords WHERE term=:t'), {'t': term}).fetchone()
    if res:
        return res[0]
    try:
        res = conn.execute(text('INSERT INTO keywords(term) VALUES(:t)'), {'t': term})
        return res.inserted_primary_key[0]
    except Exception:
        return conn.execute(text('SELECT id FROM keywords WHERE term=:t'), {'t': term}).fetchone()[0]

def insert_identifier(conn, paper_id, id_type, value):
    try:
        conn.execute(
            text('INSERT IGNORE INTO identifiers(paper_id, id_type, value) VALUES(:p, :t, :v)'),
            {'p': paper_id, 't': id_type, 'v': value}
        )
    except Exception:
        pass

def insert_section(conn, paper_id, sec_n, sec_head, sec_text):
    conn.execute(
        text('INSERT INTO sections(paper_id, sec_n, sec_head, sec_text) VALUES(:p, :n, :h, :x)'),
        {'p': paper_id, 'n': sec_n, 'h': sec_head, 'x': sec_text}
    )

def upsert_paper(conn, rec):
    res = conn.execute(
        text('SELECT id FROM papers WHERE (md5 IS NOT NULL AND md5 = :md5) OR '
             '(title <=> :title AND pub_date <=> :pub_date) ORDER BY id LIMIT 1'),
        {'md5': rec.get('md5'), 'title': rec.get('title'), 'pub_date': rec.get('pub_date')}
    ).fetchone()
    
    if res:
        paper_id = res[0]
        conn.execute(
            text('UPDATE papers SET abstract = COALESCE(NULLIF(:abstract, \'\'), abstract), '
                 'venue = COALESCE(NULLIF(:venue, \'\'), venue), '
                 'language = COALESCE(NULLIF(:lang, \'\'), language), '
                 'publisher = COALESCE(NULLIF(:publisher, \'\'), publisher) WHERE id=:id'),
            dict(abstract=rec.get('abstract') or '', venue=rec.get('venue') or '',
                 lang=rec.get('language') or '', publisher=rec.get('publisher') or '', id=paper_id)
        )
        return paper_id, False
    else:
        res = conn.execute(
            text('INSERT INTO papers(md5, tei_sha, title, abstract, venue, pub_date, language, publisher) '
                 'VALUES(:md5, NULL, :title, :abstract, :venue, :pub_date, :language, :publisher)'),
            rec
        )
        try:
            return res.inserted_primary_key[0], True
        except Exception:
            return conn.execute(
                text('SELECT id FROM papers WHERE title <=> :t ORDER BY id DESC LIMIT 1'),
                {'t': rec.get('title')}
            ).fetchone()[0], True

def ingest_tei_file(conn, path: str, verbose=False):
    data = parse_tei(path)
    data['title'] = clean_text_block(data['title'])
    data['abstract'] = clean_text_block(data['abstract'])
    
    if not data['title']:
        data['title'] = Path(path).stem
    
    pid, is_new = upsert_paper(conn, data)
    
    for id_type, value in data['ids']:
        insert_identifier(conn, pid, id_type, value)
    
    conn.execute(text('DELETE FROM paper_person WHERE paper_id=:pid'), {'pid': pid})
    for i, au in enumerate(data['authors']):
        person_id = upsert_person(conn, au.get('given'), au.get('family'), au.get('full'))
        try:
            conn.execute(
                text('INSERT IGNORE INTO paper_person(paper_id, person_id, role, seq) VALUES(:p, :r, \'author\', :s)'),
                {'p': pid, 'r': person_id, 's': i}
            )
        except Exception:
            pass
    
    conn.execute(text('DELETE FROM paper_keyword WHERE paper_id=:pid'), {'pid': pid})
    for kw in data['keywords']:
        kid = upsert_keyword(conn, kw)
        try:
            conn.execute(
                text('INSERT INTO paper_keyword(paper_id, keyword_id) VALUES(:p, :k)'),
                {'p': pid, 'k': kid}
            )
        except Exception:
            pass
    
    conn.execute(text('DELETE FROM sections WHERE paper_id=:pid'), {'pid': pid})
    for sec in data['sections']:
        insert_section(
            conn, pid, sec.get('n'),
            clean_text_block(sec.get('head')),
            clean_text_block(sec.get('text'))
        )
    
    if verbose:
        print(f"[OK] {Path(path).name} → paper_id={pid} ({'new' if is_new else 'updated'})")
    
    return pid, is_new

In [7]:
def ingest_folder(engine, tei_dir: str, pattern="*.tei.xml", verbose_every=100):
    tei_dir = Path(tei_dir)
    files = sorted(tei_dir.rglob(pattern))
    
    if not files:
        print(f"No TEI files found in {tei_dir.resolve()}")
        return 0, 0
    
    new, updated = 0, 0
    errors = []
    t0 = datetime.now()
    
    with engine.begin() as conn:
        for idx, f in enumerate(files, 1):
            try:
                pid, is_new = ingest_tei_file(conn, str(f), verbose=(verbose_every and idx % verbose_every == 0))
                if is_new:
                    new += 1
                else:
                    updated += 1
            except Exception as e:
                errors.append((str(f), repr(e)))
                if verbose_every and idx % verbose_every == 0:
                    print(f"[ERROR] {f.name} :: {repr(e)}")
    
    dt = (datetime.now() - t0).total_seconds()
    print(f"Imported {len(files)} files: {new} new, {updated} updated in {dt:.1f}s")
    
    if errors:
        print(f"{len(errors)} files failed. Sample errors:")
        for fname, err in errors[:5]:
            print(f"  {Path(fname).name}: {err}")
    
    return new, updated

new, updated = ingest_folder(engine, TEI_DIR, verbose_every=100)

[OK] A-XAI-BASED-FRAMEWORK-FOR-FREQUENCY-SUBBAND-CHARACTERIZATION-OF-COUGH-SPECTROGRAMS-IN-CHRONIC-RESPIRATORY-DISEASE-A-PREPRINT.tei.xml → paper_id=100 (new)
[OK] AN-EFFICIENT-CONDITIONAL-SCORE-BASED-FILTER-FOR-HIGH-DIMENSIONAL-NONLINEAR-FILTERING-PROBLEMS.tei.xml → paper_id=200 (new)
[OK] Autiverse-Eliciting-Autistic-Adolescents'-Daily-Narratives-through-AI-guided-Multimodal-Journaling.tei.xml → paper_id=300 (new)
[OK] BLIP-2-Bootstrapping-Language-Image-Pre-training-with-Frozen-Image-Encoders-and-Large-Language-Models.tei.xml → paper_id=400 (new)
[OK] Classification-errors-distort-findings-in-automated-speech-processing-examples-and-solutions-from-child-development-research.tei.xml → paper_id=500 (new)
[OK] CPCLDETECTOR-KNOWLEDGE-ENHANCEMENT-AND-ALIGNMENT-SELECTION-FOR-CHINESE-PATRONIZING-AND-CONDESCENDING-LANGUAGE-DETECTION.tei.xml → paper_id=600 (new)
[OK] Designing-Network-Design-Spaces.tei.xml → paper_id=700 (new)
[OK] Drag-Your-GAN-Interactive-Point-based-Manipulation-on-the-Ge

In [8]:
sql = """
SELECT p.id AS paper_id, p.title, s.sec_head, s.sec_text
FROM sections s
JOIN papers p ON p.id = s.paper_id
WHERE s.sec_text IS NOT NULL AND LENGTH(s.sec_text) > 30
"""

sec = pd.read_sql(sql, engine)
sec["sec_head"] = sec["sec_head"].fillna("").str.replace(r"\s+", " ", regex=True).str.strip()
sec["sec_text"] = sec["sec_text"].fillna("").str.replace(r"\s+", " ", regex=True).str.strip()

def split_to_chunks(text, max_len=1200, min_len=600, sep=r"(?<=[\.\?\!])\s+"):
    if not text:
        return []
    sents = re.split(sep, text)
    chunks, cur = [], ""
    for s in sents:
        if not s:
            continue
        if len(cur) + len(s) + 1 <= max_len:
            cur = (cur + " " + s).strip()
        else:
            if len(cur) < min_len and chunks:
                chunks[-1] = (chunks[-1] + " " + cur + " " + s).strip()
                cur = ""
            else:
                if cur:
                    chunks.append(cur)
                cur = s
    if cur:
        if len(cur) < min_len and chunks:
            chunks[-1] = (chunks[-1] + " " + cur).strip()
        else:
            chunks.append(cur)
    return chunks

rows = []
for r in tqdm(sec.itertuples(index=False), total=len(sec), desc="Chunking"):
    chunks = split_to_chunks(r.sec_text, max_len=1200, min_len=600)
    if not chunks:
        chunks = [r.sec_text] if r.sec_text else []
    for i, ch in enumerate(chunks):
        rows.append({
            "paper_id": r.paper_id,
            "title": r.title,
            "sec_head": r.sec_head,
            "chunk_idx": i,
            "text": ch
        })

chdf = pd.DataFrame(rows)
chdf = chdf[chdf["text"].str.len() > 30].reset_index(drop=True)

print(f"Created {len(chdf)} chunks from {len(sec)} sections")
chdf.to_csv(os.path.join(MODEL_DIR, "chunks_meta.csv"), index=False)

Chunking: 100%|██████████| 103994/103994 [00:01<00:00, 65631.78it/s]


Created 157484 chunks from 103994 sections


In [9]:
upload_df = chdf[['paper_id', 'chunk_idx', 'sec_head', 'text']].copy()
upload_df.columns = ['paper_id', 'chunk_index', 'section_head', 'content']

with engine.begin() as conn:
    conn.execute(text("DELETE FROM paper_chunks"))

upload_df.to_sql(
    'paper_chunks',
    con=engine,
    if_exists='append',
    index=False,
    method='multi',
    chunksize=5000
)

print(f"Uploaded {len(upload_df)} chunks to database")

Uploaded 157484 chunks to database


In [10]:
TOKEN_RE = re.compile(r"[a-zA-Z][a-zA-Z0-9\-]+")

def tokenize(text):
    if text is None:
        return []
    if isinstance(text, (list, tuple)):
        text = " ".join(map(str, text))
    try:
        text = text.lower()
    except Exception:
        text = str(text).lower()
    toks = TOKEN_RE.findall(text)
    return [t for t in toks if 2 <= len(t) <= 40]

tokens_list = [tokenize(t) for t in tqdm(chdf["text"].tolist(), desc="Tokenizing")]

Tokenizing: 100%|██████████| 157484/157484 [00:03<00:00, 48390.06it/s]


In [11]:
W2V_DIM = 200
W2V_WINDOW = 5
W2V_MIN_COUNT = 2
W2V_WORKERS = 4

w2v_chunks = Word2Vec(
    sentences=tokens_list,
    vector_size=W2V_DIM,
    window=W2V_WINDOW,
    min_count=W2V_MIN_COUNT,
    workers=W2V_WORKERS,
    sg=1
)

w2v_model_path = os.path.join(MODEL_DIR, "word2vec_chunks.model")
w2v_chunks.save(w2v_model_path)
print(f"Word2Vec model saved: vocab_size={len(w2v_chunks.wv.key_to_index)}")

def docvec_w2v(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv.key_to_index]
    if not vecs:
        return np.zeros(model.vector_size, dtype=np.float32)
    v = np.mean(vecs, axis=0).astype(np.float32)
    n = np.linalg.norm(v) + 1e-12
    return v / n

chunkvecs_w2v = np.vstack([
    docvec_w2v(toks, w2v_chunks) 
    for toks in tqdm(tokens_list, desc="Creating W2V vectors")
])

w2v_vectors_path = os.path.join(MODEL_DIR, "chunkvecs_w2v.npy")
np.save(w2v_vectors_path, chunkvecs_w2v)
print(f"W2V vectors saved: {chunkvecs_w2v.shape}")

Word2Vec model saved: vocab_size=109144


Creating W2V vectors: 100%|██████████| 157484/157484 [00:12<00:00, 12880.48it/s]


W2V vectors saved: (157484, 200)


In [12]:
LDA_TOPICS = 100
LDA_PASSES = 5
LDA_ALPHA = 'symmetric'

dictionary_chunks = Dictionary(tokens_list)
dictionary_chunks.filter_extremes(no_below=3, no_above=0.5, keep_n=100000)
corpus_bow_chunks = [dictionary_chunks.doc2bow(toks) for toks in tokens_list]

print(f"Dictionary size: {len(dictionary_chunks)}, Corpus size: {len(corpus_bow_chunks)}")

lda_chunks = LdaModel(
    corpus=corpus_bow_chunks,
    id2word=dictionary_chunks,
    num_topics=LDA_TOPICS,
    passes=LDA_PASSES,
    alpha=LDA_ALPHA,
    random_state=42
)

dict_path = os.path.join(MODEL_DIR, "lda_dictionary_chunks.pkl")
with open(dict_path, "wb") as f:
    pickle.dump(dictionary_chunks, f)

lda_model_path = os.path.join(MODEL_DIR, "lda_model_chunks.gensim")
lda_chunks.save(lda_model_path)
print(f"LDA model saved: {LDA_TOPICS} topics")

def dense_topic_vec(bow, lda_model, K):
    dist = np.zeros(K, dtype=np.float32)
    for tid, prob in lda_model.get_document_topics(bow, minimum_probability=0.0):
        if 0 <= tid < K:
            dist[tid] = prob
    n = np.linalg.norm(dist) + 1e-12
    return dist / n

chunkvecs_lda = np.vstack([
    dense_topic_vec(bow, lda_chunks, LDA_TOPICS) 
    for bow in tqdm(corpus_bow_chunks, desc="Creating LDA vectors")
])

lda_vectors_path = os.path.join(MODEL_DIR, "chunkvecs_lda.npy")
np.save(lda_vectors_path, chunkvecs_lda)
print(f"LDA vectors saved: {chunkvecs_lda.shape}")

Dictionary size: 73227, Corpus size: 157484
LDA model saved: 100 topics


Creating LDA vectors: 100%|██████████| 157484/157484 [00:37<00:00, 4191.33it/s]


LDA vectors saved: (157484, 100)


In [13]:
ALPHA_HYBRID = 0.5

def query_to_w2v_vec(q, model):
    toks = tokenize(q)
    return docvec_w2v(toks, model).reshape(1, -1)

def query_to_lda_vec(q, dictionary, lda_model, K):
    toks = tokenize(q)
    bow = dictionary.doc2bow(toks)
    return dense_topic_vec(bow, lda_model, K).reshape(1, -1)

def minmax(x):
    mn, mx = float(np.min(x)), float(np.max(x))
    if mx - mn < 1e-12:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def search_w2v_chunks(query, topk=10):
    """Tìm kiếm chỉ sử dụng mô hình Word2Vec"""
    q_w2v = query_to_w2v_vec(query, w2v_chunks)
    s_w2v = cosine_similarity(q_w2v, chunkvecs_w2v)[0]
    s_w2v = minmax(s_w2v)
    
    idx = np.argsort(-s_w2v)[:topk]
    out = chdf.iloc[idx].copy()
    out["score"] = s_w2v[idx]
    out["preview"] = out["text"].str[:500]
    
    return out.reset_index(drop=True)[
        ["paper_id", "title", "sec_head", "chunk_idx", "score", "preview"]
    ]

def search_lda_chunks(query, topk=10):
    """Tìm kiếm chỉ sử dụng mô hình LDA"""
    q_lda = query_to_lda_vec(query, dictionary_chunks, lda_chunks, LDA_TOPICS)
    s_lda = cosine_similarity(q_lda, chunkvecs_lda)[0]
    s_lda = minmax(s_lda)
    
    idx = np.argsort(-s_lda)[:topk]
    out = chdf.iloc[idx].copy()
    out["score"] = s_lda[idx]
    out["preview"] = out["text"].str[:500]
    
    return out.reset_index(drop=True)[
        ["paper_id", "title", "sec_head", "chunk_idx", "score", "preview"]
    ]

def hybrid_search_chunks(query, topk=10, alpha=ALPHA_HYBRID):
    """Tìm kiếm kết hợp cả Word2Vec và LDA"""
    q_w2v = query_to_w2v_vec(query, w2v_chunks)
    q_lda = query_to_lda_vec(query, dictionary_chunks, lda_chunks, LDA_TOPICS)
    
    s_w2v = cosine_similarity(q_w2v, chunkvecs_w2v)[0]
    s_lda = cosine_similarity(q_lda, chunkvecs_lda)[0]
    
    s_w2v = minmax(s_w2v)
    s_lda = minmax(s_lda)
    score = alpha * s_w2v + (1 - alpha) * s_lda
    
    idx = np.argsort(-score)[:topk]
    out = chdf.iloc[idx].copy()
    out["score"] = score[idx]
    out["score_w2v"] = s_w2v[idx]
    out["score_lda"] = s_lda[idx]
    out["preview"] = out["text"].str[:500]
    
    return out.reset_index(drop=True)[
        ["paper_id", "title", "sec_head", "chunk_idx", "score", "score_w2v", "score_lda", "preview"]
    ]

In [14]:
query = "neural networks and deep learning"
results = hybrid_search_chunks(query, topk=10, alpha=0.5)

print(f"Query: '{query}'")
print(f"Results: {len(results)} chunks\n")
print(f"Score range: {results['score'].min():.4f} - {results['score'].max():.4f}")
print(f"Mean score: {results['score'].mean():.4f}")
print(f"W2V score range: {results['score_w2v'].min():.4f} - {results['score_w2v'].max():.4f}")
print(f"LDA score range: {results['score_lda'].min():.4f} - {results['score_lda'].max():.4f}\n")

for idx, row in results.iterrows():
    print(f"[{idx+1}] Score: {row['score']:.4f} | Paper ID: {row['paper_id']}")
    print(f"    Title: {row['title'][:80]}...")
    print(f"    Section: {row['sec_head']}")
    print(f"    Preview: {row['preview'][:150]}...\n")

Query: 'neural networks and deep learning'
Results: 10 chunks

Score range: 0.8272 - 0.8940
Mean score: 0.8547
W2V score range: 0.8052 - 1.0000
LDA score range: 0.7524 - 0.9227

[1] Score: 0.8940 | Paper ID: 2274
    Title: Published as a conference paper at ICLR 2015 ADAM: A METHOD FOR STOCHASTIC OPTIM...
    Section: 6. 3 Figure 2 :Figure 3 :
    Preview: 323 Figure 2: Training of multilayer neural networks on MNIST images. (a) Neural networks using dropout stochastic regularization. (b) Neural networks...

[2] Score: 0.8809 | Paper ID: 2910
    Title: Towards End-to-End Speech Recognition with Recurrent Neural Networks...
    Section: Figure 3 .
    Preview: 3 Figure 3. Deep Recurrent Neural Network....

[3] Score: 0.8780 | Paper ID: 1403
    Title: HYBRID SPEECH RECOGNITION WITH DEEP BIDIRECTIONAL LSTM...
    Section: Fig. 3 .Fig. 4 .
    Preview: 34 Fig. 3. Deep Recurrent Neural Network...

[4] Score: 0.8639 | Paper ID: 2053
    Title: Offline Goal-conditioned Reinforcement Learni

In [15]:
def load_models(model_dir=MODEL_DIR):
    chdf_loaded = pd.read_csv(os.path.join(model_dir, "chunks_meta.csv"))
    w2v_loaded = Word2Vec.load(os.path.join(model_dir, "word2vec_chunks.model"))
    lda_loaded = LdaModel.load(os.path.join(model_dir, "lda_model_chunks.gensim"))
    with open(os.path.join(model_dir, "lda_dictionary_chunks.pkl"), "rb") as f:
        dict_loaded = pickle.load(f)
    chunkvecs_w2v_loaded = np.load(os.path.join(model_dir, "chunkvecs_w2v.npy"))
    chunkvecs_lda_loaded = np.load(os.path.join(model_dir, "chunkvecs_lda.npy"))
    
    return {
        'chunks_meta': chdf_loaded,
        'w2v_model': w2v_loaded,
        'lda_model': lda_loaded,
        'dictionary': dict_loaded,
        'w2v_vectors': chunkvecs_w2v_loaded,
        'lda_vectors': chunkvecs_lda_loaded
    }

In [16]:
chdf_loaded = pd.read_csv(os.path.join(MODEL_DIR, "chunks_meta.csv"))
w2v_loaded = Word2Vec.load(os.path.join(MODEL_DIR, "word2vec_chunks.model"))
lda_loaded = LdaModel.load(os.path.join(MODEL_DIR, "lda_model_chunks.gensim"))
with open(os.path.join(MODEL_DIR, "lda_dictionary_chunks.pkl"), "rb") as f:
    dict_loaded = pickle.load(f)
chunkvecs_w2v_loaded = np.load(os.path.join(MODEL_DIR, "chunkvecs_w2v.npy"))
chunkvecs_lda_loaded = np.load(os.path.join(MODEL_DIR, "chunkvecs_lda.npy"))

chdf = chdf_loaded
w2v_chunks = w2v_loaded
lda_chunks = lda_loaded
dictionary_chunks = dict_loaded
chunkvecs_w2v = chunkvecs_w2v_loaded
chunkvecs_lda = chunkvecs_lda_loaded
LDA_TOPICS = lda_loaded.num_topics

print(f"Models loaded: {len(chdf)} chunks, W2V vocab={len(w2v_chunks.wv.key_to_index)}, LDA topics={LDA_TOPICS}")

Models loaded: 157484 chunks, W2V vocab=109144, LDA topics=100


In [17]:
query = "transformer architecture attention mechanism"
results = hybrid_search_chunks(query, topk=10, alpha=0.5)

print(f"Query: '{query}'")
print(f"Results: {len(results)} chunks\n")
print(f"Score range: {results['score'].min():.4f} - {results['score'].max():.4f}")
print(f"Mean score: {results['score'].mean():.4f}\n")

for idx, row in results.iterrows():
    print(f"[{idx+1}] Score: {row['score']:.4f} | Paper ID: {row['paper_id']}")
    print(f"    Title: {row['title'][:80]}...")
    print(f"    Section: {row['sec_head']}")
    print(f"    Preview: {row['preview'][:150]}...\n")

Query: 'transformer architecture attention mechanism'
Results: 10 chunks

Score range: 0.8653 - 0.9522
Mean score: 0.9086

[1] Score: 0.9522 | Paper ID: 283
    Title: ATTENTIVE CONTEXTUAL CARRYOVER FOR MULTI-TURN END-TO-END SPOKEN LANGUAGE UNDERST...
    Section: Fig. 4 .
    Preview: 4 Fig. 4. Architecture of Gated Multi-head attentions....

[2] Score: 0.9515 | Paper ID: 781
    Title: Do Vision Transformers See Like Convolutional Neural Networks?...
    Section: Discussion
    Preview: Attention 1 MLP 1 Attention 2 MLP 2 Attention 3 MLP Attention 4 MLP 4 Attention 5 MLP 5 Attention 6 MLP Attention 7 MLP 7 Attention 8 MLP 8 Attention ...

[3] Score: 0.9378 | Paper ID: 283
    Title: ATTENTIVE CONTEXTUAL CARRYOVER FOR MULTI-TURN END-TO-END SPOKEN LANGUAGE UNDERST...
    Section: IVA Dataset Syn-Multi Dataset Statistic RNN-T SLU T-T SLU RNN-T SLU T-T SLU Audio encoder network
    Preview: # Layers 5 6 4 2 Layer embed-size 736 256 640 256 # Attention heads -...

[4] Score: 0.9220 | Pape